## RNN
This model is a network with 3 reccurent layers followed by a fully connected layer. We found the GRUs worked better than LSTMs, besides resulting in faster train times. We arrived at these parameters through [hyperparameter-tuning](hyperparameter-tuning_mitbih.py).

![Network](rnn.png)

In [7]:
import pandas as pd
import numpy as np

import tensorflow
from tensorflow.keras import optimizers, losses, activations, models
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Input, Dropout, GRU, \
    concatenate, Add, Activation
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import plot_model
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix

from IPython.display import Image

train = True

### Import data

In [2]:
df_train = pd.read_csv("../data/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("../data/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

### Load model and skip to [evaluation](#evaluate)

In [3]:
train = False
model = load_model("rnn_mitbih.h5")

In [6]:
img = plot_model(model, to_file="rnn.png", show_shapes=True)

### Or ALTERNATIVELY train it

In [8]:
seq_len = 187

def get_model():
    n_class = 5
    inp = Input(shape=(187, 1))
    x = GRU(128,
           return_sequences=True,
           dropout=0.2)(inp)
    x = GRU(128,
           return_sequences=True,
           dropout=0.2)(x)
    x = GRU(128,
           return_sequences=False,
           dropout=0.2)(x)
    
    dense = Dense(64, activation='relu')(x)
    dense = Dropout(0.2)(dense)
    output = Dense(n_class, activation='softmax')(dense)
    
    opt = tensorflow.keras.optimizers.Adam(lr=0.001)
    
    model = models.Model(inputs=inp, outputs=output)
    model.compile(
        loss='sparse_categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy'],
    )
    return model

In [10]:
if train:
    model = get_model()
    file_path = "rnn_mitbih_retrain.h5"
    checkpoint = ModelCheckpoint(file_path, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    early = EarlyStopping(monitor="val_accuracy", mode="max", patience=5, verbose=1)
    redonplat = ReduceLROnPlateau(monitor="val_accuracy", mode="max", patience=3, verbose=2)
    callbacks_list = [checkpoint, early, redonplat]  # early

    model.fit(X, Y, epochs=1000, verbose=2, callbacks=callbacks_list, validation_split=0.1)
    model.load_weights(file_path)

Train on 78798 samples, validate on 8756 samples
Epoch 1/1000
78798/78798 - 133s - loss: 0.5000 - accuracy: 0.8684 - val_loss: 0.2689 - val_accuracy: 0.9331
Epoch 2/1000


KeyboardInterrupt: 

### Evaluation
<a id='evaluate'></a>

In [8]:
pred_test = model.predict(X_test)
pred_test = np.argmax(pred_test, axis=-1)

f1 = f1_score(Y_test, pred_test, average="macro")

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy score : %s "% acc)

print(confusion_matrix(Y_test, pred_test))

Test f1 score : 0.9028542162135643 
Test accuracy score : 0.9835099579755162 
[[18027    58    15     7    11]
 [  118   423    13     0     2]
 [   40     4  1370    23    11]
 [   27     0    16   119     0]
 [   14     0     2     0  1592]]
